## 모듈 임포트 및 데이터 로드


 #### 구글 드라이브와 Colab을 연동합니다



In [0]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os


if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

import shutil
         


def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree("/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/딥러닝/kobert/002_bert_morp_tensorflow", "bert")

In [0]:
os.listdir('bert')

['vocab.korean_morp.list',
 'src_tokenizer',
 'bert_model.ckpt.index',
 'bert_config.json',
 'bert_model.ckpt.meta',
 'KorBERT_FAQ_20190619.pdf',
 'bert_model.ckpt.data-00000-of-00001']

- Colab에서 Bert 모형을 가지고 있는 폴더를 지정해줍니다  

- 파이썬 명령어 os.listdir(path)를 활용하여 폴더가 잘 있는지 확인하면서 진행합니다.

In [0]:
path = "/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/딥러닝"

텐서플로우, 판다스, 넘파이, 케라스 등 필요한 모듈들을 임포트합니다

In [0]:
%tensorflow_version 1.x
import os
import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

AttributeError: ignored

케라스에서 Bert 활용을 쉽게 만들어주는 모듈 keras-bert를 설치합니다<br>그리고 Adam optimizer의 수정판인 keras-radam 모듈을 임포트합니다.

In [0]:
!pip install keras-bert
!pip install keras-radam

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=8457108eda5df8cfc5484fac8b2cd0ec8201f0fd24a28ed5d8295561b1f0f255
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=e4a85a4a8d8b846691fae0a1e8e0ca2a9777e3110be4fade3d7a698940721841
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=6521b9df6b96a53b4c5c43e58b3dcf4c61d4b43d94822ffdb206e36b240e58e7
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=6b6616e42e86efff3a447808288cc911f563ca1162126e224c434aed0891148f
  Stor

keras-bert 라이브러리에서 버트 모형 활용에 필요한 모듈들을 임포트합니다

In [0]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam

Using TensorFlow backend.


In [0]:
os.listdir('bert')

['vocab.korean_morp.list',
 'src_tokenizer',
 'bert_model.ckpt.index',
 'bert_config.json',
 'bert_model.ckpt.meta',
 'KorBERT_FAQ_20190619.pdf',
 'bert_model.ckpt.data-00000-of-00001']

In [0]:
df = pd.read_csv('/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/딥러닝/glowpick_end.csv')

In [0]:
df['rating'].unique()

array([4, 3, 5, 2, 1])

In [0]:
#to_categorical 함수는 max값으로 카테고리 몇개있는지 인식하고 0부터 인식하기 때문에, 1~5있으면 6개 category있는 걸로 오해함.
df['label'] = df['rating']-1

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
#stratified
y = df['label']
X = df.drop(['label','rating'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=99, shuffle=True, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99, shuffle=True, stratify=y_train)

In [0]:
train = pd.concat([X_train, y_train], axis=1).reset_index()
test = pd.concat([X_val, y_val], axis=1).reset_index()

## 모델 사전 설정 및 단계별 형태소 분석


- bert 훈련을 위한 사전 설정을 합니다. SEQ_LEN은 문장의 최대 길이입니다. SEQ_LEN 보다 문장의 길이가 작다면 남은 부분은 0이 채워지고, 만약에 SEQ_LEN보다 문장 길이가 길다면 SEQ_LEN을 초과하는 부분이 잘리게 됩니다.  
- BATCH_SIZE는 메모리 초과 같은 문제를 방지하기 위해 작은 수인 16으로 정했습니다. 그리고 총 훈련 에포크 수는 2로 정했습니다. 학습율(LR;Learning rate)은 1e-5로 작게 정했습니다.  
- pretrained_path는 bert 사전학습 모형이 있는 폴더를 의미합니다.
- 그리고 우리가 분석할 문장이 들어있는 칼럼의 제목인 document와 긍정인지 부정인지 알려주는 칼럼을 label로 정해줍니다


In [0]:
import os
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS=2
LR=1e-5

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
#vocab_path = os.path.join(pretrained_path, 'vocab.korean_morp.list')

DATA_COLUMN = "fin_review"
LABEL_COLUMN = "label"

In [0]:
# LIBRARIES

import collections       # OrderedDict를 위해 호출
import re                # 정규표현식
import unicodedata       # 한국어 정준분해 및 문자열 확인
import six               # Python version 체크
import tensorflow as tf  # Tensorflow 파일 불러오기 및 logging

In [0]:
# ARGUMENTS

# ETRI에서 받은 file path를 저장
path2 = '/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/딥러닝/kobert/002_bert_morp_tensorflow/'
path4 = '/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/딥러닝/kobert/004_bert_eojeol_tensorflow/'
# 한국어 vocab 사전을 등록
morph_vocab_file = path2 + 'vocab.korean_morp.list'
rawtext_vocab_file = path4 + 'vocab.korean.rawtext.list'

do_lower_case = True # default=False, 정준분해 예시를 위해 True로 설정
max_len = None # 없으면 1e12

텍스트 분석 패키지 로드

In [0]:
import urllib3
import logging
import json
logger = logging.getLogger(__name__)
from getpass import getpass

#from konlpy.tag import Komoran
#from konlpy.tag import Mecab

형태소 분석기 load

In [0]:
#ETRI 형태소 분석기
def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
	 
    requestJson = { "access_key": openapi_key, "argument": { "text": text, "analysis_code": "morp" } }
	 
    http = urllib3.PoolManager()
    response = http.request( "POST", openApiURL, headers={"Content-Type": "application/json; charset=UTF-8"}, body=json.dumps(requestJson))
    
    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason      
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:                        
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result

In [3]:
# 단어 사전 호출

def convert_to_unicode(text):
    # Python version이 3.x일 때,
    # type(text)이 `bytes`일 경우, utf-8로 변환
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    # Python version이 2.x일 때,
    # type(text)이 `str`일 경우, utf-8로 변환
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    # Python 3.x, 2.x만 허용!
    else:
        raise ValueError("Not running on Python2 or Python 3?")
        
        
def _load_vocab(vocab_file):
    # 단어 사전을 저장할 OrderedDict 객체 생성
    vocab = collections.OrderedDict()
    index = 0
    with tf.io.gfile.GFile(vocab_file, 'r') as reader:
        while True:
            # Binary Text를 unicode(utf-8)로 decode.
            token = convert_to_unicode(reader.readline())
            if not token: break
            if ((token.find('n_iters=') == 0) or
                (token.find('max_length=') == 0)):
                continue
            token = token.split('\t')[0]
            token = token.strip()
            # 토큰과 해당 index를 기록
            vocab[token] = index
            index += 1
    return vocab

# 단어 사전 호출
morph_vocab = _load_vocab(morph_vocab_file)
rawtext_vocab = _load_vocab(rawtext_vocab_file)

NameError: ignored

In [0]:
# 100개만 출력해보기
# 뒤의 품사에 `_`가 붙은 것을 잘 기억해두기
{key:value for i, (key, value) in enumerate(morph_vocab.items()) if i < 100}

{'"/SS_': 25,
 '%/SW_': 81,
 "'/SS_": 26,
 '(/SS_': 29,
 ')/SS_': 30,
 ',/SP_': 17,
 '-/SS_': 95,
 './SF_': 7,
 '/NNG_': 83,
 '1/SN_': 50,
 '2/SN_': 54,
 '3/SN_': 63,
 '4/SN_': 82,
 '5/SN_': 91,
 '<S>': 5,
 '<T>': 6,
 '[CLS]': 2,
 '[MASK]': 4,
 '[PAD]': 0,
 '[SEP]': 3,
 '[UNK]': 1,
 '·/SP_': 75,
 '‘/SS_': 65,
 '’/SS_': 66,
 '“/SS_': 67,
 '”/SS_': 69,
 'ㄴ/ETM_': 10,
 'ㄴ다/EF_': 41,
 'ㄹ/ETM_': 35,
 '가/JKS_': 24,
 '것/NNB_': 33,
 '게/EC_': 47,
 '고/EC_': 23,
 '고/JKQ_': 64,
 '과/JC_': 46,
 '과/JKB_': 84,
 '그/NP_': 90,
 '기/ETN_': 49,
 '까지/JX_': 79,
 '년/NNB_': 39,
 '는/ETM_': 22,
 '는/JX_': 18,
 '다/EF_': 8,
 '다고/EC_': 96,
 '다는/ETM_': 89,
 '대하/VV_': 73,
 '던/ETM_': 68,
 '도/JX_': 34,
 '되/VV_': 57,
 '되/XSV_': 32,
 '들/XSN_': 36,
 '등/NNB_': 51,
 '로/JKB_': 31,
 '를/JKO_': 19,
 '만/JX_': 98,
 '만/NR_': 85,
 '말/NNG_': 72,
 '며/EC_': 53,
 '면/EC_': 71,
 '면서/EC_': 88,
 '명/NNB_': 87,
 '받/VV_': 78,
 '수/NNB_': 55,
 '아/EC_': 62,
 '않/VX_': 59,
 '았/EP_': 43,
 '어/EC_': 20,
 '어서/EC_': 94,
 '없/VA_': 70,
 '었/EP_': 12,
 '에/JK

In [0]:
# 100개만 출력해보기
# 뒤의 품사에 `_`가 붙은 것을 잘 기억해두기
{key:value for i, (key, value) in enumerate(rawtext_vocab.items()) if i < 100}

{'"': 14,
 "'": 17,
 '(': 15,
 ')': 19,
 ')_': 66,
 ',_': 8,
 '.': 42,
 '._': 7,
 '1': 52,
 '2': 65,
 '3': 71,
 '4': 99,
 '5': 98,
 '<S>': 5,
 '<T>': 6,
 '[CLS]': 2,
 '[MASK]': 4,
 '[PAD]': 0,
 '[SEP]': 3,
 '[UNK]': 1,
 '_': 9,
 '‘': 84,
 '“': 88,
 '가': 41,
 '가_': 25,
 '고': 39,
 '고_': 23,
 '공': 89,
 '과_': 30,
 '구': 93,
 '기': 37,
 '나': 85,
 '는_': 21,
 '다': 33,
 '대': 38,
 '도': 81,
 '도_': 29,
 '동': 75,
 '라': 94,
 '로': 77,
 '로_': 26,
 '를_': 32,
 '리': 47,
 '마': 72,
 '무': 69,
 '미': 87,
 '보': 60,
 '부': 55,
 '비': 73,
 '사': 35,
 '상': 64,
 '성': 80,
 '세': 83,
 '소': 79,
 '수': 40,
 '수_': 43,
 '스': 56,
 '시': 36,
 '신': 67,
 '아': 53,
 '어': 62,
 '에_': 13,
 '에서_': 22,
 '여': 97,
 '연': 91,
 '오': 86,
 '와_': 54,
 '원': 68,
 '유': 61,
 '으로_': 31,
 '은_': 18,
 '을_': 12,
 '의_': 11,
 '이': 20,
 '이_': 10,
 '이다': 63,
 '인': 51,
 '인_': 57,
 '일': 46,
 '있는_': 92,
 '있다': 28,
 '자': 48,
 '장': 70,
 '재': 95,
 '전': 44,
 '정': 49,
 '제': 76,
 '조': 74,
 '주': 45,
 '지': 27,
 '지_': 82,
 '하': 90,
 '하고_': 58,
 '하는_': 34,
 '한': 96,
 '한_

토크나이즈 함수 정의


In [0]:
def tokenize(self, text):
      split_tokens = []
      # End to End Tokenizing.
      for token in self.basic_tokenizer.tokenize(text):
          # ETRI Vocab 양식에 맞게 token 끝에 '_'를 붙여준다.
          token += '_'
          for sub_token in self.wordpiece_tokenizer.tokenize(token):
              split_tokens.append(sub_token)
      return split_tokens

In [0]:
SPLIT_TOKENS = [] # 최종적으로 return할 list 생성

In [0]:
def tokenize(self, text):
      text = convert_to_unicode(text) #1
      text = self._clean_text(text)   #2

      orig_tokens = whitespace_tokenize(text) #3
      split_tokens = []
      for token in orig_tokens:
          if self.do_lower_case:
              # 현재 input으로 '고객/NNG'와 같이 Part-of-speech가 이미
              # tagging되어있고 vocab은 '고객/NNG_'로 단어를 기록하고 있음.
              # 여기서 `lower` 메서드를 사용하면 뒤의 tagging이 소문자로
              # 변환되어 값의 비교를 못하게 되므로 이를 주석처리.

              # token.lower()

              # 모든 음절을 정준 분해시키는 함수
              token = self._run_strip_accents(token) #4
          split_tokens.extend(self._run_split_on_punc(token)) #5
      output_tokens = whitespace_tokenize(" ".join(split_tokens)) #6
      return output_tokens


## 전처리

In [0]:
#1. unicode 변환
text = convert_to_unicode(do_lang('5b085632-32e4-43b4-8ffa-a7fb3649c1bd',df[DATA_COLUMN][2]))
text

In [0]:
## Tokenize하면서 계속 사용된 character 단위 함수 정의
def _is_control(char):
    if char == "\t" or char == "\n" or char == "\r":
        # 개행문자이면 False 반환
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        # unicode category가
        # Cc(Control)
        # Cf(format)
        # Co(Private Use, is 0)
        # Cs(Surrrogate, is 0)일 경우, True 반환
        # https://en.wikipedia.org/wiki/Control_character
        return True
    # 이 외의 경우 전부 False 반환
    return False
    
def _is_whitespace(char):
    if char == " " or char == '\t' or char == '\n' or char == '\r':
        # 개행문자이거나 띄어쓰기면 True 반환
        return True
    cat = unicodedata.category(char)
    if cat == 'Zs':
        # unicode category가 Space Seperator면 True 반환
        # https://www.compart.com/en/unicode/category/Zs
        return True
    # 이 외의 경우 전부 False 반환
    return False

def _is_punctuation(char):
    # 한국어 형태소 분석기이기 때문에 공백과 같은지 여부만 반환
    return char == ' '

In [0]:
#2. text cleaning
def _clean_text(text):
    output = [] # char을 저장할 list 생성
    for char in text:
        # 텍스트에서 Char 단위로 출력
        cp = ord(char)
        if cp == 0 or cp == 0xfffd or _is_control(char):
            # \x00이거나 �이거나 unicode cat.이 C로 시작할 경우
            # (개행문자 제외) output에 추가하지 않는다.
            continue
        if _is_whitespace(char):
            # 공백일 경우 " "으로 output에 추가
            output.append(" ")
        else:
            # 이 외의 경우 전부 output에 추가
            output.append(char)
    # cleaning 작업을 거친 Text를 후처리하여 반환
    return "".join(output)

_clean_text(text) # 뭐가 변했을까? 안변한거 같지>?

'백/NR 탁/NNB 현상/NNG 없/VA 고/EC 부드럽/VA 게/EC 발려져서/NNG 아주/MAG 맘/NNG 들/VV 어요/EC '

토큰화


버트 모형은 문장 앞에 꼭 [CLS]라는 문자가 위치하고, [SEP]라는 문자가 끝에 위치합니다.  
[CLS]는 문장의 시작, [SEP]는 문장의 끝을 의미합니다.

In [0]:
#3. whitespacing(띄어쓰기로 token화)
# text 단위 공백 처리
def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip() # 양 사이드의 공백을 제거
    if not text: # 어떠한 값도 없을 시, 빈 list를 반환
        return []
    tokens = text.split() # 공백 단위로 쪼갠 list를 반환
    return tokens

orig_tokens = whitespace_tokenize(text)
orig_tokens

['백/NR',
 '탁/NNB',
 '현상/NNG',
 '없/VA',
 '고/EC',
 '부드럽/VA',
 '게/EC',
 '발려져서/NNG',
 '아주/MAG',
 '맘/NNG',
 '들/VV',
 '어요/EC']

In [0]:
split_tokens = []
# >>> 첫 번째 for loop
token = orig_tokens[0]
token # for loop 돌리기 전에 어떻게 돌아가는지 체크
# token = '심사 숙고'

'백/NR'

In [0]:
#4. 음절을 정준분해
#print(token, end=' >> ')
token = unicodedata.normalize("NFD", token)
print(list(token))
print('print(token) ==', token, '(사실 출력시에는 변화 X)')
# https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
# 모든 음절을 정준 분해(Canonical Decomposition)시킴
# '각'을 'ㄱ+ㅏ+ㄱ'으로 저장(출력되는 값은 동일)
output = []
for char in token:
    cat = unicodedata.category(char)
    if cat == "Mn":
        # unicode category가 "Mark, Nonspacing"일 경우 pass
        continue
    output.append(char)
token = ''.join(output)
token # if문에 해당하는 char가 없었기에 원본 text를 출력
      # 정준분해된 상태임을 기억해라

['ᄇ', 'ᅢ', 'ᆨ', '/', 'N', 'R']
print(token) == 백/NR (사실 출력시에는 변화 X)


'백/NR'

In [0]:
#5. punctuation 구분(사실상 의미가 없다)
chars = list(token)
i, start_new_word = 0, True
output = []
#print(chars)

In [0]:
#print('char\t_is_puntuation(char)\tstart_new_word\tOutput')
while i < len(chars):
    char = chars[i]
    #print(char, end='\t')
    #print(_is_punctuation(char), end='\t\t\t')
    if _is_punctuation(char):
        #print('In Here!! <CODE BLOCK "IF _IS_PUNCTUATOIN">')
        output.append([char])
        start_new_word = True
    else:
        if start_new_word:
            output.append([])
        start_new_word = False
        output[-1].append(char)
    #print(start_new_word, end='\t\t')
    #print(output)
    i += 1
["".join(x) for x in output]

['백/NR']

In [0]:
# for loop을 적용하면 아래와 같이 된다.
split_tokens = []
for token in orig_tokens:
    #4. 음절을 정준분해
    #print(token, end=' >> ')
    token = unicodedata.normalize("NFD", token)
    #print(list(token))
    #print('print(token) ==', token, '(사실 출력시에는 변화 X)')
    # https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
    # 모든 음절을 정준 분해(Canonical Decomposition)시킴
    # '각'을 'ㄱ+ㅏ+ㄱ'으로 저장(출력되는 값은 동일)
    output = []
    for char in token:
        cat = unicodedata.category(char)
        if cat == "Mn":
            # unicode category가 "Mark, Nonspacing"일 경우 pass
            continue
        output.append(char)
    token = ''.join(output) # if문에 해당하는 char가 없었기에 원본 text를 출력
                            # 정준분해된 상태임을 기억해라
    #5. punctuation 구분(사실상 의미가 없다)
    chars = list(token)
    i, start_new_word = 0, True
    output = []
    print(chars)
    print('char\t_is_puntuation(char)\tstart_new_word\tOutput')
    while i < len(chars):
        char = chars[i]
        #print(char, end='\t')
        #print(_is_punctuation(char), end='\t\t\t')
        if _is_punctuation(char):
            #print('In Here!! <CODE BLOCK "IF _IS_PUNCTUATOIN">')
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        #print(start_new_word, end='\t\t')
        #print(output)
        i += 1
    split_tokens.extend(["".join(x) for x in output])
#print('split_tokens:', split_tokens)
output_tokens = whitespace_tokenize(" ".join(split_tokens))
print('output_tokens:', output_tokens)

['ᄇ', 'ᅢ', 'ᆨ', '/', 'N', 'R']
char	_is_puntuation(char)	start_new_word	Output
['ᄐ', 'ᅡ', 'ᆨ', '/', 'N', 'N', 'B']
char	_is_puntuation(char)	start_new_word	Output
['ᄒ', 'ᅧ', 'ᆫ', 'ᄉ', 'ᅡ', 'ᆼ', '/', 'N', 'N', 'G']
char	_is_puntuation(char)	start_new_word	Output
['ᄋ', 'ᅥ', 'ᆹ', '/', 'V', 'A']
char	_is_puntuation(char)	start_new_word	Output
['ᄀ', 'ᅩ', '/', 'E', 'C']
char	_is_puntuation(char)	start_new_word	Output
['ᄇ', 'ᅮ', 'ᄃ', 'ᅳ', 'ᄅ', 'ᅥ', 'ᆸ', '/', 'V', 'A']
char	_is_puntuation(char)	start_new_word	Output
['ᄀ', 'ᅦ', '/', 'E', 'C']
char	_is_puntuation(char)	start_new_word	Output
['ᄇ', 'ᅡ', 'ᆯ', 'ᄅ', 'ᅧ', 'ᄌ', 'ᅧ', 'ᄉ', 'ᅥ', '/', 'N', 'N', 'G']
char	_is_puntuation(char)	start_new_word	Output
['ᄋ', 'ᅡ', 'ᄌ', 'ᅮ', '/', 'M', 'A', 'G']
char	_is_puntuation(char)	start_new_word	Output
['ᄆ', 'ᅡ', 'ᆷ', '/', 'N', 'N', 'G']
char	_is_puntuation(char)	start_new_word	Output
['ᄃ', 'ᅳ', 'ᆯ', '/', 'V', 'V']
char	_is_puntuation(char)	start_new_word	Output
['ᄋ', 'ᅥ', 'ᄋ', 'ᅭ', '/', 'E', 'C']
char	_is_p

In [0]:

output_tokens = [token + '_' for token in output_tokens]
output_tokens # ETRI 단어 사전에 맞게 form을 변경

['백/NR_',
 '탁/NNB_',
 '현상/NNG_',
 '없/VA_',
 '고/EC_',
 '부드럽/VA_',
 '게/EC_',
 '발려져서/NNG_',
 '아주/MAG_',
 '맘/NNG_',
 '들/VV_',
 '어요/EC_']

In [0]:
unk_token = '[UNK]'
max_input_chars_per_word = 128

In [0]:
vocab = morph_vocab

In [0]:
SPLIT_TOKENS = []
for text in output_tokens:
    text = convert_to_unicode(text)
    _output_tokens = []
    # whitespacing 생략
    chars = list(text)
    if len(chars) > max_input_chars_per_word:
        _output_tokens.append(unk_token)
    is_bad = False
    start = 0
    sub_tokens = []
    while start < len(chars):
        end = len(chars)
        cur_substr = None
        while start < end:
            substr = "".join(chars[start:end])
            substr = unicodedata.normalize("NFC", substr)
            if substr in vocab:
                cur_substr = substr
                break
            end -= 1
            #print(substr)
        if cur_substr is None:
            is_bad = True
            break
        sub_tokens.append(cur_substr)
        start = end
    if is_bad:
        _output_tokens.append(unk_token)
    else:
        _output_tokens.extend(sub_tokens)
    
    for sub_token in _output_tokens:
        SPLIT_TOKENS.append(sub_token)

In [0]:

max_seq_length = 128
len(SPLIT_TOKENS) > max_seq_length - 2

False

In [0]:
SPLIT_TOKENS

['백/NR_',
 '탁',
 '/NNB_',
 '현상/NNG_',
 '없/VA_',
 '고/EC_',
 '부드럽/VA_',
 '게/EC_',
 '발',
 '려',
 '져',
 '서/NNG_',
 '아주/MAG_',
 '맘/NNG_',
 '들/VV_',
 '어요/EC_']

In [0]:
# The convention in BERT is:
# (a) For sequence pairs:
# tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
# type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
# (b) For single sequences:
# tokens:   [CLS] the dog is hairy . [SEP]
# type_ids: 0     0   0   0  0     0 0
#
# Where "type_ids" are used to indicate whether this is the first
# sequence or the second sequence. The embedding vectors for 'type=0' and
# 'type=1' were learned during pre-training and are added to the wordpiece
# embedding vector (and position vector). This is not "strictly" necessary
# since the [SEP] token unambigiously separates the sequences, but it makes
# if easier for the model to learn the concept of sequences.
#
# For classification tasks, the first vector (corresponding to [CLS]) is
# used as the "sentence vector". Note that this only makes sense because
# the entire model is fine-tuned.

tokens = ["[CLS]"] + SPLIT_TOKENS + ["[SEP]"]
segment_ids = [0] * len(tokens)

In [0]:
import numpy as np
print(np.vstack((tokens, [morph_vocab[token] for token in tokens])))

[['[CLS]' '백/NR_' '탁' '/NNB_' '현상/NNG_' '없/VA_' '고/EC_' '부드럽/VA_' '게/EC_'
  '발' '려' '져' '서/NNG_' '아주/MAG_' '맘/NNG_' '들/VV_' '어요/EC_' '[SEP]']
 ['2' '3819' '3874' '811' '1874' '70' '23' '6160' '47' '479' '2100'
  '6650' '354' '2925' '6690' '261' '7807' '3']]


In [0]:

segment_ids

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
input_ids = [morph_vocab[token] for token in tokens]
print(input_ids)

[2, 3819, 3874, 811, 1874, 70, 23, 6160, 47, 479, 2100, 6650, 354, 2925, 6690, 261, 7807, 3]


In [0]:
# The mask has 1 for real tokens and 0 for padding tokens. Only real
# tokens are attended to.
input_mask = [1] * len(input_ids)
input_mask

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [0]:
# Zero-pad up to the sequence length.
padding = [0] * (max_seq_length - len(input_ids))
input_ids += padding
input_mask += padding
segment_ids += padding

In [0]:
assert len(input_ids) == max_seq_length
assert len(input_mask) == max_seq_length
assert len(segment_ids) == max_seq_length

In [0]:
input_ids

[2,
 3819,
 3874,
 811,
 1874,
 70,
 23,
 6160,
 47,
 479,
 2100,
 6650,
 354,
 2925,
 6690,
 261,
 7807,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [0]:
from keras.utils import to_categorical
import array


In [13]:
'error' in do_lang('5b085632-32e4-43b4-8ffa-a7fb3649c1bd','졸리다')

True

In [21]:
35000 % 5000 == 0

True

글로우픽 데이터를 **버트 모형의 입력에 맞게 변형해주는 함수**를 정의합니다.

1. 전처리
2. 토큰화
3. 재토큰화(wordpiece)
4. 인덱싱

In [0]:
vocab = morph_vocab
unk_token = '[UNK]'
SEQ_LEN = 128
max_input_chars_per_word = 128
openapi_key = '5b085632-32e4-43b4-8ffa-a7fb3649c1bd'

def convert_data(data_df):
    global tokenizer
    indices, segments, targets = [], [], []
    for i in tqdm(range(len(data_df))):
        targets.append(data_df[LABEL_COLUMN][i])
       
        if len(data_df) % 5000 == 0:
          openapi_key = input('apikey변경필요! 입력요망 :')
          text = do_lang(openapi_key,data_df[DATA_COLUMN][i])
        else:
          text = do_lang(openapi_key,data_df[DATA_COLUMN][i])
        text = convert_to_unicode(text)
        text = _clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        
        
        split_tokens = []
        for token in orig_tokens:
          #4. 음절을 정준분해
          #print(token, end=' >> ')
          token = unicodedata.normalize("NFD", token)
          #print(list(token))
          #print('print(token) ==', token, '(사실 출력시에는 변화 X)')
          # https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
          # 모든 음절을 정준 분해(Canonical Decomposition)시킴
          # '각'을 'ㄱ+ㅏ+ㄱ'으로 저장(출력되는 값은 동일)
          output = []
          for char in token:
            cat = unicodedata.category(char)
            if cat == "Mn":
              # unicode category가 "Mark, Nonspacing"일 경우 pass
              continue
            output.append(char)
          token = ''.join(output) # if문에 해당하는 char가 없었기에 원본 text를 출력
                            # 정준분해된 상태임을 기억해라
          #5. punctuation 구분(사실상 의미가 없다)
          chars = list(token)
          i, start_new_word = 0, True
          output = []
          #print(chars)
          #print('char\t_is_puntuation(char)\tstart_new_word\tOutput')
          while i < len(chars):
            char = chars[i]
            #print(char, end='\t')
            #print(_is_punctuation(char), end='\t\t\t')
            if _is_punctuation(char):
              #print('In Here!! <CODE BLOCK "IF _IS_PUNCTUATOIN">')
              output.append([char])
              start_new_word = True
            else:
              if start_new_word:
                output.append([])
              start_new_word = False
              output[-1].append(char)
            #print(start_new_word, end='\t\t')
            #print(output)
            i += 1
          split_tokens.extend(["".join(x) for x in output])
        #print('split_tokens:', split_tokens)
        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        #print('output_tokens:', output_tokens)
        output_tokens = [token + '_' for token in output_tokens]
        SPLIT_TOKENS = []
        for text in output_tokens:
          text = convert_to_unicode(text)
          _output_tokens = []
          # whitespacing 생략
          chars = list(text)
          if len(chars) > max_input_chars_per_word:
            _output_tokens.append(unk_token)
          is_bad = False
          start = 0
          sub_tokens = []
          while start < len(chars):
            end = len(chars)
            cur_substr = None
            while start < end:
              substr = "".join(chars[start:end])
              substr = unicodedata.normalize("NFC", substr)
              if substr in vocab:
                cur_substr = substr
                break
              end -= 1
              #print(substr)
            if cur_substr is None:
              is_bad = True
              break
            sub_tokens.append(cur_substr)
            start = end
          if is_bad:
            _output_tokens.append(unk_token)
          else:
            _output_tokens.extend(sub_tokens)
    
          for sub_token in _output_tokens:
            SPLIT_TOKENS.append(sub_token)
          tokens = ["[CLS]"] + SPLIT_TOKENS + ["[SEP]"]
          if len(tokens) > max_seq_length -2:
            new_msl = max_seq_length -2 
            tokens = tokens[0:new_msl]
            segment_ids = [0] * len(tokens)
            input_ids = [morph_vocab[token] for token in tokens]
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            segment_ids += padding
          else:
            segment_ids = [0] * len(tokens)
            input_ids = [morph_vocab[token] for token in tokens]
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            segment_ids += padding
          
        indices.append(input_ids)
        segments.append(segment_ids)
    items = list(zip(indices, targets))
    indices, targets = zip(*items)
    targets = to_categorical(targets,5)
    
              
    return [np.vstack(indices), np.vstack(segments)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
     
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x, data_y= convert_data(data_df)

    return data_x, data_y

In [0]:
train_x, train_y = load_data(train)
test_x, test_y = load_data(test)

100%|██████████| 10718/10718 [58:28<00:00,  3.05it/s]


In [0]:
train_x

- 사전학습된 버트 모델의 인풋은 문장 토큰화가 숫자로 바뀐 것과, 앞문장인지 뒷문장인지 알려주는 문장 순서 벡터가 들어갑니다. 우리는 문장 하나를 가지고만 훈련할 것이므로 순서 벡터는 모두 0으로 통일합니다.

- 그리고 파인튜닝 시에는 문장 안에 일부 단어를 가리는 마스킹은 사용하지 않습니다.

In [0]:
len(train_x[0][0])

128

In [0]:
train_y

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

## 모델 fine-tuning

**버트 모형에 들어갈 인풋은 토큰, 세그먼트, 포지션으로 구성됩니다.**  
버트에 인풋으로 들어가는 토큰은 문장을 토크나이징 한 후, 인덱스 번호를 매긴 것입니다.  
세그먼트는 예를 들어 문장이 두 개가 있다면, 앞의 문장과 뒤의 문장을 구분하는 것입니다.  
포지션 임베딩은 단순히 단어의 위치를 말합니다.

토큰, 세그먼트, 포지션을 인풋으로 버트 모형에 넣으면 기하학적인 문장 공간으로 임베딩이 됩니다.

![대체 텍스트](https://i.imgur.com/l9BTao3.png)

#### 이해가 안 가실 수 있는데, 버트 인풋을 문장으로 예를 들어 만들어 보겠습니다.
#### 인풋은 총 2개가 들어갑니다
- **(토큰)** 첫번째 인풋은 토큰화 된 것이 인덱싱되어 숫자로 변환된 것  

- **(세그멘트)** 두번째 인풋은 앞문장인지 뒷문장인지 알려주는 숫자들입니다. 이번 튜토리얼에서는 파인튜닝 과정이라 앞문장 뒷문장 구분을 안하기 때문에 모두 0으로 하였습니다.  

- **(포지션)** 단어 순서에 따라서 자동으로 부여됩니다.


이해가 되셨는지요?  
구글 깃허브에서 다운받았던 사전학습된 모델을 colab으로 로드합니다.

In [0]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

In [0]:
train_y.shape

(32154, 5)

모델의 구조를 확인합니다.  
총 12층의 트랜스포머 계층이 있음을 확인할 수 있습니다.


In [0]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 128)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 23308032    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

가장 중요한 부분인데요, 사전학습 파일을 로드하여, 우리가 불러들였던 사전학습 모델을 변형해줍니다.  
  

input으로는 우리가 문장을 토큰화 하여 숫자로 변형시켜주었던 **토큰 벡터와**, 앞문장인지 뒷문장인지 알려주는 **세그멘트** 두 가지가 되겠습니다.  
  
즉 인풋은 inputs = modle.inputs[:2]로 정의하였습니다.  
  
  
그리고 output은 일단 사전학습 모델을 약간 잘라줍니다.(outputs=Dense(1)) 맨 위 3층을 잘라 낸다음에 잘라낸 부분에 긍정인지 부정인지 알려주는 **Dense(1)을 사전학습 모델에 애드온 시켜 줍니다.**  

Dense(5)은 아웃풋이 5개, 각 라벨의 확률을 나타냅니다


그리고 사전 설치하였던 Radam을 활용하여 deep learning의 기울기 강하 훈련을 하도록 정해줍니다. 그 다음에 bert_model을 return해 줍니다.

In [0]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output


  outputs = keras.layers.Dense(5, activation='softmax',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss='categorical_crossentropy',
      metrics=['accuracy'])
  
  return bert_model
  

**모델의 FLOW를 확인해 보도록 하겠습니다.**

In [0]:
from IPython.display import SVG
from keras.utils import model_to_dot


SVG(model_to_dot(get_bert_finetuning_model(model), dpi=65).create(prog='dot', format='svg'))


bert_model을 get_bert_finetuning_model 함수로 불러들입니다.
이 다음에 bert_model.fit을 활용하여 버트 모형 파인튜닝을 시작하게 됩니다.


BiLSTM 이나 어텐션 기법을 사용했을 시 보통 테스트셋에 대한 정확도가 86% 정도로 알려져 있습니다.

In [0]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=5, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)

In [0]:
import matplotlib.pyplot as plt

In [0]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.title('model accuracy')
plt.show()

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.title('model loss')
plt.show()

재사용을 위해 bert_model을 지드라이브에 저장해줍니다.

In [0]:
bert_model.save_weights(path+"/korbert_preprocessed_final_epoch5.h5")

버트 모형을 로드해줍니다. 이미 로드하였던 모델에 계수들만 살짝 얹혀 줍니다.

In [0]:
bert_model = get_bert_finetuning_model(model)
bert_model.load_weights(path+"/korbert_preprocessed_final_epoch5.h5")

파인튜닝한 버트 모형에 test 데이터 셋을 넣어 평가해 봅니다.
사실 머신러닝에서는 정확도인 accuracy도 중요하지만, **F1 score**가 상당히 중요합니다.
  

긍정문장으로 판별된 문장이 진짜로 **긍정 문장에 속할 확률**  
  

부정문장으로 판별된 문장이 진짜로 **부정 문장에 속할 확률**  
 **이 두가지를 고려한 종합적인 성능이 F1 score입니다.**

한번 F1-score를 확인해 보겠습니다.


먼저 테스트 데이터를 버트 모형에 넣을 수 있도록 predict_convert_data 함수를 정의해줍니다. <br> 위에 정의한 convert_data 함수와 다른 점은, test 데이터이기 때문에 label은 고려하지 않는다는 점입니다

## 모델 평가

In [0]:
vocab = morph_vocab
unk_token = '[UNK]'
SEQ_LEN = 128
max_input_chars_per_word = 128
openapi_key = '5b085632-32e4-43b4-8ffa-a7fb3649c1bd'

def predict_convert_data(data_df):
    global tokenizer
    indices, segments= [], []
    for i in tqdm(range(len(data_df))):
        
        if len(data_df) % 5000 == 0:
          openapi_key = input('apikey변경필요! 입력요망 :')
          text = do_lang(openapi_key,data_df[DATA_COLUMN][i])
        else:
          text = do_lang(openapi_key,data_df[DATA_COLUMN][i])
        text = convert_to_unicode(text)
        text = _clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        
        
        split_tokens = []
        for token in orig_tokens:
          #4. 음절을 정준분해
          #print(token, end=' >> ')
          token = unicodedata.normalize("NFD", token)
          #print(list(token))
          #print('print(token) ==', token, '(사실 출력시에는 변화 X)')
          # https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
          # 모든 음절을 정준 분해(Canonical Decomposition)시킴
          # '각'을 'ㄱ+ㅏ+ㄱ'으로 저장(출력되는 값은 동일)
          output = []
          for char in token:
            cat = unicodedata.category(char)
            if cat == "Mn":
              # unicode category가 "Mark, Nonspacing"일 경우 pass
              continue
            output.append(char)
          token = ''.join(output) # if문에 해당하는 char가 없었기에 원본 text를 출력
                            # 정준분해된 상태임을 기억해라
          #5. punctuation 구분(사실상 의미가 없다)
          chars = list(token)
          i, start_new_word = 0, True
          output = []
          #print(chars)
          #print('char\t_is_puntuation(char)\tstart_new_word\tOutput')
          while i < len(chars):
            char = chars[i]
            #print(char, end='\t')
            #print(_is_punctuation(char), end='\t\t\t')
            if _is_punctuation(char):
              #print('In Here!! <CODE BLOCK "IF _IS_PUNCTUATOIN">')
              output.append([char])
              start_new_word = True
            else:
              if start_new_word:
                output.append([])
              start_new_word = False
              output[-1].append(char)
            #print(start_new_word, end='\t\t')
            #print(output)
            i += 1
          split_tokens.extend(["".join(x) for x in output])
        #print('split_tokens:', split_tokens)
        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        #print('output_tokens:', output_tokens)
        output_tokens = [token + '_' for token in output_tokens]
        SPLIT_TOKENS = []
        for text in output_tokens:
          text = convert_to_unicode(text)
          _output_tokens = []
          # whitespacing 생략
          chars = list(text)
          if len(chars) > max_input_chars_per_word:
            _output_tokens.append(unk_token)
          is_bad = False
          start = 0
          sub_tokens = []
          while start < len(chars):
            end = len(chars)
            cur_substr = None
            while start < end:
              substr = "".join(chars[start:end])
              substr = unicodedata.normalize("NFC", substr)
              if substr in vocab:
                cur_substr = substr
                break
              end -= 1
              #print(substr)
            if cur_substr is None:
              is_bad = True
              break
            sub_tokens.append(cur_substr)
            start = end
          if is_bad:
            _output_tokens.append(unk_token)
          else:
            _output_tokens.extend(sub_tokens)
    
          for sub_token in _output_tokens:
            SPLIT_TOKENS.append(sub_token)
          tokens = ["[CLS]"] + SPLIT_TOKENS + ["[SEP]"]
          if len(tokens) > max_seq_length -2:
            new_msl = max_seq_length -2 
            tokens = tokens[0:new_msl]
            segment_ids = [0] * len(tokens)
            input_ids = [morph_vocab[token] for token in tokens]
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            segment_ids += padding
          else:
            segment_ids = [0] * len(tokens)
            input_ids = [morph_vocab[token] for token in tokens]
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            segment_ids += padding
          
        indices.append(input_ids)
        segments.append(segment_ids)


              
    return [np.vstack(indices), np.vstack(segments)]

def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
     
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x= predict_convert_data(data_df)

    return data_x

In [0]:
test = pd.concat([X_test, y_test], axis=1).reset_index()

In [0]:
test_set = predict_load_data(test)

In [0]:
test_set

## 테스트 셋으로 예측하기

In [0]:
#예측
preds = bert_model.predict(test_set)

In [0]:
# 부정이면 0, 긍정이면 1 출력
preds

In [0]:
from sklearn.metrics import classification_report


In [0]:
preds_list = []
for i in range(0, len(preds)):
  preds_list.append([np.where(preds[i] == a) for a in preds[i] if a == max(preds[i])][0][0][0]) 

In [0]:
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, preds_list, digits=5))

F1 스코어와 Accuracy가 거의 비슷함을 알 수 있습니다. 훈련이 치우치지 않고 잘 됐음을 알 수 있습니다.

## 한번 문장을 입력하면 긍정인지 부정인지 알려주는 함수를 만들어 볼까요?

In [0]:
openapi_key = '5b085632-32e4-43b4-8ffa-a7fb3649c1bd'
def sentence_convert_data(sentence):
    global tokenizer
    indices, segments= [], []
    
    sentence = do_lang(openapi_key,data_df[DATA_COLUMN][i])   
    text = convert_to_unicode(sentence)
    text = _clean_text(text)
    orig_tokens = whitespace_tokenize(text)
        
        
    split_tokens = []
    for token in orig_tokens:
      #4. 음절을 정준분해
      #print(token, end=' >> ')
      token = unicodedata.normalize("NFD", token)
      #print(list(token))
      #print('print(token) ==', token, '(사실 출력시에는 변화 X)')
      # https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
      # 모든 음절을 정준 분해(Canonical Decomposition)시킴
      # '각'을 'ㄱ+ㅏ+ㄱ'으로 저장(출력되는 값은 동일)
      output = []
      for char in token:
        cat = unicodedata.category(char)
        if cat == "Mn":
          # unicode category가 "Mark, Nonspacing"일 경우 pass
          continue
        output.append(char)
      token = ''.join(output) # if문에 해당하는 char가 없었기에 원본 text를 출력
                            # 정준분해된 상태임을 기억해라
      #5. punctuation 구분(사실상 의미가 없다)
      chars = list(token)
      i, start_new_word = 0, True
      output = []
      #print(chars)
      #print('char\t_is_puntuation(char)\tstart_new_word\tOutput')
      while i < len(chars):
        char = chars[i]
        #print(char, end='\t')
        #print(_is_punctuation(char), end='\t\t\t')
        if _is_punctuation(char):
          #print('In Here!! <CODE BLOCK "IF _IS_PUNCTUATOIN">')
          output.append([char])
          start_new_word = True
        else:
          if start_new_word:
            output.append([])
          start_new_word = False
          output[-1].append(char)
        #print(start_new_word, end='\t\t')
        #print(output)
        i += 1
      split_tokens.extend(["".join(x) for x in output])
    #print('split_tokens:', split_tokens)
    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    #print('output_tokens:', output_tokens)
    output_tokens = [token + '_' for token in output_tokens]
    SPLIT_TOKENS = []
    for text in output_tokens:
      text = convert_to_unicode(text)
      _output_tokens = []
      # whitespacing 생략
      chars = list(text)
      if len(chars) > max_input_chars_per_word:
        _output_tokens.append(unk_token)
      is_bad = False
      start = 0
      sub_tokens = []
      while start < len(chars):
        end = len(chars)
        cur_substr = None
        while start < end:
          substr = "".join(chars[start:end])
          substr = unicodedata.normalize("NFC", substr)
          if substr in vocab:
            cur_substr = substr
            break
          end -= 1
          #print(substr)
        if cur_substr is None:
          is_bad = True
          break
        sub_tokens.append(cur_substr)
        start = end
      if is_bad:
        _output_tokens.append(unk_token)
      else:
        _output_tokens.extend(sub_tokens)
    
      for sub_token in _output_tokens:
        SPLIT_TOKENS.append(sub_token)
      tokens = ["[CLS]"] + SPLIT_TOKENS + ["[SEP]"]
      if len(tokens) > max_seq_length -2:
        new_msl = max_seq_length -2 
        tokens = tokens[0:new_msl]
        segment_ids = [0] * len(tokens)
        input_ids = [morph_vocab[token] for token in tokens]
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        segment_ids += padding
      else:
        segment_ids = [0] * len(tokens)
        input_ids = [morph_vocab[token] for token in tokens]
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        segment_ids += padding
          
    indices.append(input_ids)
    segments.append(segment_ids)


              
    return [np.vstack(indices), np.vstack(segments)]

def cosmetic_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = bert_model.predict(data_x)
    
    
    predict_answer = [np.where(predict[0] == a) for a in predict[0] if a == max(predict[0])][0][0][0]
    if predict_answer == 0:
      print("1점")
    elif predict_answer == 1:
      print("2점")
    elif predict_answer == 2:
      print("3점")
    elif predict_answer == 3:
      print("4점")
    else:
      print("5점")


In [0]:
cosmetic_evaluation_predict("부드럽게 발려서 좋았어요")

In [0]:
cosmetic_evaluation_predict("분사력이 별로에요")

In [0]:
cosmetic_evaluation_predict("홈쇼핑에서다량으로산걸후회중인울마미ㅜㅜ") 

In [0]:
cosmetic_evaluation_predict("향이 무엇보다 너무 맘에 안들어서 안쓰게 되던")

In [0]:
cosmetic_evaluation_predict("한통 깔끔히 다쓰고 재구매하면서 주변 사람들한테 추천하는중")